In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from public_tests import *

plt.rcParams['figure.figsize']=(5.0,4.0)
plt.rcParams['image.interpolation']='nearest'
plt.rcParams['image.cmap']='gray'
np.random.seed(1)

> ### Notations 
- Superscript [l] denotes an object of the lth layer
   - eg: a[4] is the 4th layer activation ..W[5] and b[5] are the 5th layer parameters.

- superscript(i) denotes an object from the ith eg.
   - eg: x(i) is the ith training eg input
   
- subscript i denote the ith entry of a vector.
    - eg: ai[l] denotes the ith entry of the activation in layer l, assuming this is a fully connected layer
   

> ### steps of cnn

- convolution functions, including:
    - zero padding
    - convolve window
    - convolution forward
    - convolution bacward

- pooling functions:
    - pooling forward
    - create mask
    - Distribute mask
    - pooling backward
    

In [3]:
#define padding ..zero padding

def zero_pad(X,pad):
    X_pad=np.pad(X,((0,0),(pad,pad),(pad,pad),(0,0)),mode='constant',constant_values=(0,0))
    
    return X_pad

In [11]:
# #checking 
# np.random.seed(1)
# x=np.random.randn(4,3,3,2)
# x_pad=zero_pad(x,2)
# print('x.shape=\n',x_pad.shape)
# print('x[1,1]=\n',x[1,1])
# print('x_pad[1,1]=\n',x_pad[1,1])


# fig,axarr=plt.subplot(1, 2)
# axarr[0].set_title('x')
# axarr[0].imshow(x[0, :, :, 0])
# axarr[1].set_title('x_pad')
# axarr[1].imshow(x_pad[0, :, :, 0])
# zero_pad_test(zero_pad)

In [6]:
def conv_single_step(a_slice_prev,W,b):
    
    s=np.multiply(a_slice_prev,W)
    Z=np.sum(s)
    Z=Z+float(b)
    return Z